# Step 2: Generate data samples

In this section, we will use the amplitude model that we created with the
expert system to generate a data sample via hit & miss Monte Carlo. This
requires us to work with the User Interface to ComPWA, the C++ back-end of
`pycompwa`.

## 2.1 Generate

In the previous step, we created an amplitude module and exported it as an
XML file. We now need to go through a few steps to generate a phase space
sample and a data sample for the decay $J/\psi \rightarrow \gamma\pi^0\pi^0$:

Now all building blocks for generating our data sample are at hand and you
can define a data sample of arbitrary size!

In [1]:
import pycompwa.ui as pwa  # interface to ComPWA

2021-10-22 22:33:20,258 INFO [default] Logging to file disabled!
2021-10-22 22:33:20,259 [INFO] Log level: INFO
2021-10-22 22:33:20,259 [INFO] Current date and time: Fri Oct 22 22:33:20 2021



/opt/hostedtoolcache/Python/3.7.12/x64/lib/python3.7/site-packages/traitlets/traitlets.py:3050: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  FutureWarning,


In [2]:
# specify kinematics
particle_list = pwa.read_particles("model.xml")
kinematics = pwa.create_helicity_kinematics("model.xml", particle_list)

# specify generators
generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info()
)
random_generator = pwa.StdUniformRealGenerator(12345)

# generate phase space sample
phsp_sample = pwa.generate_phsp(int(1e5), generator, random_generator)

# create intensity profile
intensity_builder = pwa.IntensityBuilderXML(
    "model.xml", particle_list, kinematics, phsp_sample
)
intensity = intensity_builder.create_intensity()

# generate data
data_sample = pwa.generate(
    int(1e4), kinematics, generator, intensity, random_generator
)

2021-10-22 22:33:20,616 [INFO] HelicityKinematics::HelicityKinematics() | Initialized kinematics for reaction ( J/psi )->( gamma[ID=2] pi0[ID=3] pi0[ID=4] )
Event position to final state ID mapping:
0: 2
1: 3
2: 4



2021-10-22 22:33:20,618 [INFO] Generating phase-space MC: [100000 events] 


 0.00min [ ---------- 0.00% ] 0.00min , end time: 10:33PM               
 0.00min [ ********** 100.00% ] 0.00min , end time: 10:33PM               

2021-10-22 22:33:20,714 [INFO] Setting phase space sample weights...


2021-10-22 22:33:20,722 [INFO] Updating data container content...


2021-10-22 22:33:22,624 [INFO] Generating hit-and-miss sample: [10000 events] 


 0.00min [ ---------- 0.00% ] 0.00min , end time: 10:33PM               


2021-10-22 22:33:23,247 [INFO] Tools::generate() | Error in HitMiss procedure: Maximum value of random number generation smaller then amplitude maximum! We raise the maximum to 9.92631 value and restart generation!


 0.00min [ ---------- 0.00% ] 0.00min , end time: 10:33PM               


 0.03min [ ********** 100.00% ] 0.00min , end time: 10:33PM               2021-10-22 22:33:25,403 [INFO] Successfully generated 10000 with an efficiency of 0.0740741


## 2.2 Exporting and importing

In [3]:
# to ROOT
pwa.write_root_data(data_sample, output_file="generated_data.root")
pwa.write_root_data(phsp_sample, output_file="generated_phsp.root")
# to ASCII
pwa.write_ascii_data(data_sample, output_file="generated_data.dat")
pwa.write_ascii_data(phsp_sample, output_file="generated_phsp.dat")

ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!


Invoking:
  

LC_ALL=C x86_64-linux-gnu-g++-7  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 >/dev/null | awk '/^#include </,/^End of search/{if (!/^#include </ && !/^End of search/){ print }}' | GREP_OPTIONS= grep -E "(c|g)\+\+"

Results was:


With exit code 

256

2021-10-22 22:33:25,610 [INFO] Root::writeData(): writing vector of 10000 events to file generated_data.root


2021-10-22 22:33:25,744 [INFO] Root::writeData(): writing vector of 100000 events to file generated_phsp.root


2021-10-22 22:33:26,287 [INFO] Logging to file disabled!


The syntax for importing parallels that:

In [4]:
imported_data = pwa.read_root_data(input_file="generated_data.root")
imported_phsp = pwa.read_ascii_data(input_file="generated_phsp.dat")

print("Imported data events:", len(imported_data.events))
print("Imported phsp events:", len(imported_phsp.events))

first_event = imported_data.events[0]
print("First data event:")
print(first_event.four_momenta)
print(first_event.weight)

Imported data events: 10000
Imported phsp events: 100000
First data event:
FourMomentumList[(-0.520903,0.885259,1.21872), (0.653672,-0.813022,1.46359), (-0.132769,-0.0722372,0.414596)]
1.0


Note that the ASCII (``.dat``) files contain a header section that informs
you about the final state of the file. You will have to prepend this section
to the file yourself if you want to use an ASCII file that was exported from
another framework.

In [5]:
with open("generated_data.dat") as f:
    for _ in range(5):
        print(f.readline()[:-1])

<header>
	Pids: [22, 111, 111]
	Order: px py pz E
	Unit: GeV
</header>
